In [9]:
import json
import pandas as pd
import numpy as np
import os

def files_to_df(dir_name):
    
    full_df = pd.DataFrame() # initialize full df compilation of files
    files = os.listdir(dir_name) # get list of files in directory
    print(f"{len(files)} files found in {dir_name}")

    for i in files:
        filename = os.path.join(dir_name, i)
        with open(filename, 'r') as f:
            data = json.loads(json.load(f))['data']
            df = pd.DataFrame(data)
        full_df = pd.concat([full_df, df], ignore_index = True) # join current json file to full df

    return full_df


def df_to_json(query):
    
    print(f"Converting json files in raw_data/{query} to dataframe.")
    df = files_to_df(f'raw_data/{query}')
    print(f"Shape of dataframe: {df.shape}")
        
    # convert dataframe back to json and export it
    export = df.to_json(f"full_data/{query}.json")
    print("Export complete.")
    
    return df

In [10]:
import time
import requests

def pause_report(length, random_delay, file_count, disp = True):
    
    pause_time = length*np.random.normal(1,random_delay)
    if disp: print(f"Downloaded {file_count} images. Pausing scraper for {round(pause_time,2)} seconds.")
    time.sleep(pause_time)

    
def download_files(query, df, file_start, file_end, short_pause = 2, long_pause = 60, random_delay = 0.25):

    file_count = 0 #number of files downloaded
    alrdy_exists = 0 #number of files that already exist and are skipped over
    skipped_urls = {} #urls that can't be connected to - need to retry at a later time
    
    if not os.path.exists(f'images/{query}'):
        print(f'Creating directory: images/{query}')
        os.makedirs(f'images/{query}')
    
    for i in range(file_start, file_end):

        if i >= len(df):
            print(f"Index {i} does not exist in dataframe.")
            print("End of dataframe has probably been reached. Process is terminating.")
            return 0
        
        filename = f"images/{query}/{df['hash_id'][i]}.jpg" # name the image using its hash_id

        if not os.path.exists(filename): # skip over files that have been downloaded
            
            try:
                img_url = df['smaller_square_cover_url'][i]
                print(f'Downloading {img_url} as {filename}')
                
                f = open(filename,'wb')
                f.write(requests.get(img_url).content)
                f.close()

                file_count += 1

                # take a short pause per url scraped
                pause_report(short_pause, random_delay, file_count)
                
            except:
                print("Cannot establish connection with the following row in df:")
                print(f"{i}:{img_url}")
                print("Skipping to next row")
                skipped_urls[i] = img_url #add key value pair to skipped_urls dict
                
                if len(skipped_urls) > 10:
                    print("More than 10 urls have had connection error.")
                    print(skipped_urls)
                    pause_report(300, random_delay, file_count) # pause for 5 minutes if more than 10 urls with connection error
                       
        else: 
            print(f"{filename} already exists. Skipping to next image url in df.")
            alrdy_exists += 1
            
        # take a long pause if 100 images have been downloaded
        if file_count > 0 and file_count%100 == 0:
            pause_report(long_pause, random_delay, file_count)
            
        # take a long pause x 2 if 500 images have been downloaded
        if file_count > 0 and file_count%500 == 0:
            pause_report(long_pause*2, random_delay, file_count)
      
    # Retry skipped urls
    print(f"Skipped the following urls because failed to make a connection:")
    print(skipped_urls)
    
    if len(skipped_urls) > 0:
        print("Retrying skipped urls.")
        
        for index, url in skipped_urls.items():
            try:
                filename = f"images/{query}/{df['hash_id'][index]}.jpg"
                print(f'Downloading {url} as {filename}.')

                f = open(filename,'wb')
                f.write(requests.get(img_url).content)
                f.close()

                file_count += 1

                # take a short pause per url scraped
                pause_report(short_pause, random_delay, file_count)

            except Exception as e:
                print("Still unable to connect to skipped urls. Please manually check for error.")
                print(e.message, e.args)
                return 0
    
    print(f"{file_count} images downloaded.")
    print(f"Skipped {alrdy_exists} images because already exists in database.")
    print("Process finished.")
    return 1
    

In [11]:
genres = ['cyberpunk','noir','horror','western','cartoon','steampunk']

df = df_to_json(genres[2])

Converting json files in raw_data/horror to dataframe.
133 files found in raw_data/horror
Shape of dataframe: (9975, 8)
Export complete.


In [13]:
download_files(genres[2], df, 4900, 10000)

images/horror/lVdabG.jpg already exists. Skipping to next image url in df.
images/horror/N502AJ.jpg already exists. Skipping to next image url in df.
images/horror/XV0Ja.jpg already exists. Skipping to next image url in df.
images/horror/Era6q.jpg already exists. Skipping to next image url in df.
images/horror/dXk4A.jpg already exists. Skipping to next image url in df.
images/horror/VV2Qg.jpg already exists. Skipping to next image url in df.
images/horror/4bbPlk.jpg already exists. Skipping to next image url in df.
images/horror/mNQqd.jpg already exists. Skipping to next image url in df.
images/horror/oJOx4.jpg already exists. Skipping to next image url in df.
images/horror/WbZeJ.jpg already exists. Skipping to next image url in df.
images/horror/azdJz.jpg already exists. Skipping to next image url in df.
images/horror/AzZKm.jpg already exists. Skipping to next image url in df.
images/horror/r6Yq2.jpg already exists. Skipping to next image url in df.
images/horror/e0xVN6.jpg already ex

Downloaded 4 images. Pausing scraper for 2.13 seconds.
Downloaded 5 images. Pausing scraper for 1.55 seconds.
images/horror/mqqLBd.jpg already exists. Skipping to next image url in df.
Downloaded 6 images. Pausing scraper for 1.75 seconds.
Downloaded 7 images. Pausing scraper for 2.02 seconds.
Downloaded 8 images. Pausing scraper for 1.94 seconds.
Downloaded 9 images. Pausing scraper for 2.67 seconds.
Downloaded 10 images. Pausing scraper for 1.88 seconds.
Downloaded 11 images. Pausing scraper for 0.59 seconds.
Downloaded 12 images. Pausing scraper for 1.34 seconds.
Downloaded 13 images. Pausing scraper for 1.34 seconds.
Downloaded 14 images. Pausing scraper for 2.04 seconds.
Downloaded 15 images. Pausing scraper for 1.82 seconds.
Downloaded 16 images. Pausing scraper for 2.88 seconds.
Downloaded 17 images. Pausing scraper for 2.1 seconds.
Downloaded 18 images. Pausing scraper for 1.59 seconds.
Downloaded 19 images. Pausing scraper for 2.23 seconds.
Downloaded 20 images. Pausing scrape

Downloaded 40 images. Pausing scraper for 1.68 seconds.
Downloaded 41 images. Pausing scraper for 1.83 seconds.
Downloaded 42 images. Pausing scraper for 1.37 seconds.
Downloaded 43 images. Pausing scraper for 2.33 seconds.
Downloaded 44 images. Pausing scraper for 1.9 seconds.
Downloaded 45 images. Pausing scraper for 2.0 seconds.
Downloaded 46 images. Pausing scraper for 1.5 seconds.
Downloaded 47 images. Pausing scraper for 2.42 seconds.
Downloaded 48 images. Pausing scraper for 2.42 seconds.
Downloaded 49 images. Pausing scraper for 2.01 seconds.
Downloaded 50 images. Pausing scraper for 2.19 seconds.
Downloaded 51 images. Pausing scraper for 1.45 seconds.
Downloaded 52 images. Pausing scraper for 2.23 seconds.
Downloaded 53 images. Pausing scraper for 2.62 seconds.
Downloaded 54 images. Pausing scraper for 2.09 seconds.
Downloaded 55 images. Pausing scraper for 1.95 seconds.
Downloaded 56 images. Pausing scraper for 1.62 seconds.
Downloaded 57 images. Pausing scraper for 2.39 seco

Downloaded 77 images. Pausing scraper for 1.98 seconds.
Downloaded 78 images. Pausing scraper for 2.61 seconds.
Downloaded 79 images. Pausing scraper for 2.54 seconds.
Downloaded 80 images. Pausing scraper for 2.4 seconds.
Downloaded 81 images. Pausing scraper for 1.9 seconds.
Downloaded 82 images. Pausing scraper for 1.41 seconds.
Downloaded 83 images. Pausing scraper for 1.28 seconds.
Downloaded 84 images. Pausing scraper for 1.55 seconds.
Downloaded 85 images. Pausing scraper for 1.98 seconds.
Downloaded 86 images. Pausing scraper for 2.41 seconds.
Downloaded 87 images. Pausing scraper for 1.76 seconds.
Downloaded 88 images. Pausing scraper for 2.38 seconds.
Downloaded 89 images. Pausing scraper for 2.64 seconds.
Downloaded 90 images. Pausing scraper for 1.19 seconds.
Downloaded 91 images. Pausing scraper for 2.14 seconds.
Downloaded 92 images. Pausing scraper for 1.02 seconds.
Downloaded 93 images. Pausing scraper for 1.72 seconds.
Downloaded 94 images. Pausing scraper for 2.64 sec

Downloaded 113 images. Pausing scraper for 1.94 seconds.
Downloaded 114 images. Pausing scraper for 2.49 seconds.
Downloaded 115 images. Pausing scraper for 2.13 seconds.
Downloaded 116 images. Pausing scraper for 2.66 seconds.
Downloaded 117 images. Pausing scraper for 2.23 seconds.
Downloaded 118 images. Pausing scraper for 1.84 seconds.
Downloaded 119 images. Pausing scraper for 2.74 seconds.
Downloaded 120 images. Pausing scraper for 1.79 seconds.
Downloaded 121 images. Pausing scraper for 1.95 seconds.
Downloaded 122 images. Pausing scraper for 1.59 seconds.
Downloaded 123 images. Pausing scraper for 1.94 seconds.
Downloaded 124 images. Pausing scraper for 1.95 seconds.
Downloaded 125 images. Pausing scraper for 2.02 seconds.
Downloaded 126 images. Pausing scraper for 2.24 seconds.
Downloaded 127 images. Pausing scraper for 2.55 seconds.
Downloaded 128 images. Pausing scraper for 1.28 seconds.
Downloaded 129 images. Pausing scraper for 2.37 seconds.
Downloaded 130 images. Pausing 

...

Downloaded 4820 images. Pausing scraper for 2.3 seconds.
Downloaded 4821 images. Pausing scraper for 2.68 seconds.
Downloaded 4822 images. Pausing scraper for 2.6 seconds.
Downloaded 4823 images. Pausing scraper for 1.53 seconds.
Downloaded 4824 images. Pausing scraper for 2.23 seconds.
Downloaded 4825 images. Pausing scraper for 1.68 seconds.
Downloaded 4826 images. Pausing scraper for 1.58 seconds.
Downloaded 4827 images. Pausing scraper for 2.02 seconds.
Downloaded 4828 images. Pausing scraper for 2.66 seconds.
Downloaded 4829 images. Pausing scraper for 1.88 seconds.
Downloaded 4830 images. Pausing scraper for 2.17 seconds.
Downloaded 4831 images. Pausing scraper for 1.0 seconds.
Downloaded 4832 images. Pausing scraper for 2.81 seconds.
Downloaded 4833 images. Pausing scraper for 0.99 seconds.
Downloaded 4834 images. Pausing scraper for 1.58 seconds.
Downloaded 4835 images. Pausing scraper for 2.21 seconds.
Downloaded 4836 images. Pausing scraper for 1.97 seconds.
Downloaded 4837 i

Downloaded 4855 images. Pausing scraper for 2.39 seconds.
Downloaded 4856 images. Pausing scraper for 1.61 seconds.
Downloaded 4857 images. Pausing scraper for 2.89 seconds.
Downloaded 4858 images. Pausing scraper for 1.92 seconds.
Downloaded 4859 images. Pausing scraper for 2.32 seconds.
Downloaded 4860 images. Pausing scraper for 1.07 seconds.
Downloaded 4861 images. Pausing scraper for 1.9 seconds.
Downloaded 4862 images. Pausing scraper for 1.28 seconds.
Downloaded 4863 images. Pausing scraper for 2.18 seconds.
Downloaded 4864 images. Pausing scraper for 1.27 seconds.
Downloaded 4865 images. Pausing scraper for 1.91 seconds.
Downloaded 4866 images. Pausing scraper for 2.64 seconds.
Downloaded 4867 images. Pausing scraper for 2.08 seconds.
Downloaded 4868 images. Pausing scraper for 1.88 seconds.
Downloaded 4869 images. Pausing scraper for 2.0 seconds.
Downloaded 4870 images. Pausing scraper for 2.2 seconds.
Downloaded 4871 images. Pausing scraper for 2.1 seconds.
Downloaded 4872 im

Downloaded 4891 images. Pausing scraper for 2.16 seconds.
Downloaded 4892 images. Pausing scraper for 3.0 seconds.
Downloaded 4893 images. Pausing scraper for 1.61 seconds.
Downloaded 4894 images. Pausing scraper for 2.38 seconds.
Downloaded 4895 images. Pausing scraper for 2.25 seconds.
Downloaded 4896 images. Pausing scraper for 1.89 seconds.
Downloaded 4897 images. Pausing scraper for 2.21 seconds.
Downloaded 4898 images. Pausing scraper for 1.16 seconds.
Downloaded 4899 images. Pausing scraper for 2.22 seconds.
Downloaded 4900 images. Pausing scraper for 1.8 seconds.
Downloaded 4900 images. Pausing scraper for 55.12 seconds.
Downloaded 4901 images. Pausing scraper for 2.29 seconds.
Downloaded 4902 images. Pausing scraper for 1.27 seconds.
Downloaded 4903 images. Pausing scraper for 2.41 seconds.
Downloaded 4904 images. Pausing scraper for 1.63 seconds.
Downloaded 4905 images. Pausing scraper for 3.16 seconds.
Downloaded 4906 images. Pausing scraper for 1.67 seconds.
Downloaded 4907

Downloaded 4926 images. Pausing scraper for 2.31 seconds.
Downloaded 4927 images. Pausing scraper for 2.04 seconds.
Downloaded 4928 images. Pausing scraper for 1.76 seconds.
Downloaded 4929 images. Pausing scraper for 2.31 seconds.
Downloaded 4930 images. Pausing scraper for 1.81 seconds.
Downloaded 4931 images. Pausing scraper for 1.88 seconds.
Downloaded 4932 images. Pausing scraper for 1.66 seconds.
Downloaded 4933 images. Pausing scraper for 2.99 seconds.
Downloaded 4934 images. Pausing scraper for 1.8 seconds.
Downloaded 4935 images. Pausing scraper for 1.51 seconds.
Downloaded 4936 images. Pausing scraper for 1.73 seconds.
Downloaded 4937 images. Pausing scraper for 2.11 seconds.
Downloaded 4938 images. Pausing scraper for 2.17 seconds.
Downloaded 4939 images. Pausing scraper for 1.59 seconds.
Downloaded 4940 images. Pausing scraper for 2.89 seconds.
Downloaded 4941 images. Pausing scraper for 1.82 seconds.
Downloaded 4942 images. Pausing scraper for 1.59 seconds.
Downloaded 4943

0

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

plt.figure(figsize=(20,6))
for i in range(5):
    plt.subplot(1,5,i+1)
    img = mpimg.imread(f"images/{query}/{df['hash_id'][i+60]}.jpg")
    plt.title(f"{df['title'][i]}")
    plt.imshow(img)
    
    

In [14]:
grades = [3.38,3.03,3.69,3.18,3.74,3.94,3.29,3.91,3.67,3.35,4,3.67,3.85,4,3.32,3.77,3.73,3.67,3.67,3.82,3.5,3.96,4.22,3.99]
np.mean(grades)


3.68125